In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
Independent=dataset[['Gender_Male','Age','EstimatedSalary']]
Independent

,Gender_Male,Age,EstimatedSalary
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000
...,...,...,...
395,0,46,41000
396,1,51,23000
397,0,50,20000
398,1,36,33000


In [8]:
Dependent=dataset[['Purchased']]
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [10]:
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
param_grid={'alpha':[1,5,10,100],'force_alpha':[True,False],'fit_prior':[True,False]}
grid=GridSearchCV(ComplementNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [1, 5, 10, 100], 'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(x_test) #(.predict->defined function)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[42 37]
 [20 21]]


In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.68      0.53      0.60        79
           1       0.36      0.51      0.42        41

    accuracy                           0.53       120
   macro avg       0.52      0.52      0.51       120
weighted avg       0.57      0.52      0.54       120



In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("f1_macro value for {}:".format(grid.best_params_),f1_macro)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

f1_macro value for {'alpha': 1, 'fit_prior': True, 'force_alpha': True}: 0.5371480765097786


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.017686,0.008581,0.029931,0.003572,1,True,True,"{'alpha': 1, 'fit_prior': True, 'force_alpha':...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
1,0.017950,0.006142,0.030058,0.004477,1,True,False,"{'alpha': 1, 'fit_prior': True, 'force_alpha':...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
2,0.011291,0.004120,0.021454,0.008340,1,False,True,"{'alpha': 1, 'fit_prior': False, 'force_alpha'...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
3,0.012154,0.001380,0.017592,0.001875,1,False,False,"{'alpha': 1, 'fit_prior': False, 'force_alpha'...",0.527697,0.472527,0.540136,0.578065,0.558458,0.535377,0.035740,1
4,0.009088,0.005165,0.020118,0.004868,5,True,True,"{'alpha': 5, 'fit_prior': True, 'force_alpha':...",0.507692,0.488602,0.540136,0.578065,0.541831,0.531265,0.030843,5
5,0.009407,0.002600,0.012550,0.003494,5,True,False,"{'alpha': 5, 'fit_prior': True, 'force_alpha':...",0.507692,0.488602,0.540136,0.578065,0.541831,0.531265,0.030843,5
6,0.013090,0.002772,0.010786,0.003866,5,False,True,"{'alpha': 5, 'fit_prior': False, 'force_alpha'...",0.507692,0.488602,0.540136,0.578065,0.541831,0.531265,0.030843,5
7,0.007959,0.003867,0.016406,0.004829,5,False,False,"{'alpha': 5, 'fit_prior': False, 'force_alpha'...",0.507692,0.488602,0.540136,0.578065,0.541831,0.531265,0.030843,5
8,0.010830,0.004924,0.009780,0.003859,10,True,True,"{'alpha': 10, 'fit_prior': True, 'force_alpha'...",0.507692,0.469075,0.540136,0.578065,0.541831,0.527360,0.036681,9
9,0.008427,0.002399,0.010529,0.005988,10,True,False,"{'alpha': 10, 'fit_prior': True, 'force_alpha'...",0.507692,0.469075,0.540136,0.578065,0.541831,0.527360,0.036681,9
